In [1]:
import cv2
import tensorflow as tf
from sklearn import datasets, model_selection, metrics
import open3d as o3d
import numpy as np
import os
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
import time

# Pedir el nombre del objeto
object_name = input("Introduce el nombre del objeto: ")

# Crear la carpeta si no existe
if not os.path.exists(object_name):
    os.makedirs(object_name)

# Iniciar la cámara con alta resolución
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # Ajustar resolución a Full HD
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

image_count = len(os.listdir(object_name))  # Contar las imágenes en la carpeta
taking_photos = False  # Variable para controlar si se están tomando fotos

cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Frame', 640, 480)  # Ajustar la ventana a 640 x 480

while True:
    ret, frame = cap.read()
    
    # Mostrar el frame resultante
    cv2.imshow('Frame', frame)

    # Verificar si se presionó la tecla 'e' para comenzar a tomar fotos
    key = cv2.waitKey(1) & 0xFF
    if key == ord('e'):
        taking_photos = True
        start_time = time.time()

    # Verificar si se presionó la tecla 'r' para dejar de tomar fotos
    if key == ord('r'):
        taking_photos = False

    # Tomar ráfagas de 10 fotos cada 2 segundos si está activado
    if taking_photos and time.time() - start_time >= 2:
        for _ in range(10):
            ret, frame = cap.read()
            if ret:
                image_path = f"{object_name}/image_{image_count}.jpg"
                cv2.imwrite(image_path, frame, [cv2.IMWRITE_JPEG_QUALITY, 100])  # Guardar con calidad 100%
                print(f"Imagen guardada: {image_path}")
                image_count += 1
        start_time = time.time()  # Reiniciar el temporizador
    
    # Salir con la tecla 'q'
    if key == ord('q'):
        break

# Liberar la cámara y cerrar ventanas
cap.release()
cv2.destroyAllWindows()


Imagen guardada: audifonos/image_0.jpg
Imagen guardada: audifonos/image_1.jpg
Imagen guardada: audifonos/image_2.jpg
Imagen guardada: audifonos/image_3.jpg
Imagen guardada: audifonos/image_4.jpg
Imagen guardada: audifonos/image_5.jpg
Imagen guardada: audifonos/image_6.jpg
Imagen guardada: audifonos/image_7.jpg
Imagen guardada: audifonos/image_8.jpg
Imagen guardada: audifonos/image_9.jpg
Imagen guardada: audifonos/image_10.jpg
Imagen guardada: audifonos/image_11.jpg
Imagen guardada: audifonos/image_12.jpg
Imagen guardada: audifonos/image_13.jpg
Imagen guardada: audifonos/image_14.jpg
Imagen guardada: audifonos/image_15.jpg
Imagen guardada: audifonos/image_16.jpg
Imagen guardada: audifonos/image_17.jpg
Imagen guardada: audifonos/image_18.jpg
Imagen guardada: audifonos/image_19.jpg
Imagen guardada: audifonos/image_20.jpg
Imagen guardada: audifonos/image_21.jpg
Imagen guardada: audifonos/image_22.jpg
Imagen guardada: audifonos/image_23.jpg
Imagen guardada: audifonos/image_24.jpg
Imagen gua

In [3]:
# Resize images to a smaller resolution
def resize_images(images, size=(224, 224)):
    resized_images = []
    for img in images:
        resized_img = cv2.resize(img, size)
        resized_images.append(resized_img)
    return np.array(resized_images)

# Resize the training images
X_train_resized = resize_images(X_train)

# Fit the ImageDataGenerator on the resized images
datagen.fit(X_train_resized)


In [7]:
# Resize images to a smaller resolution
def resize_images(images, size=(224, 224)):
    resized_images = []
    for img in images:
        resized_img = cv2.resize(img, size)
        resized_images.append(resized_img)
    return np.array(resized_images)

# Resize the training and test images
X_train_resized = resize_images(X_train_resized)
X_test_resized = resize_images(X_test)

# Fit the ImageDataGenerator on the resized images
datagen.fit(X_train_resized)

# Crear el modelo
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(X_train_resized.shape[1], X_train_resized.shape[2], X_train_resized.shape[3])),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),  # Regularización con Dropout
    Dense(len(lb.classes_), activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo con más épocas y un tamaño de lote más pequeño
history = model.fit(
    datagen.flow(X_train_resized, y_train, batch_size=16),  # Reducir el tamaño del lote
    epochs=50,  # Entrenar por 50 épocas
    validation_data=(X_test_resized, y_test)
)

# Guardar el modelo
model.save('object_recognition_model_2.h5')

print("Modelo entrenado y guardado correctamente.")

Epoch 1/50
21/21 [==============================] - 4s 184ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/50
21/21 [==============================] - 4s 209ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/50
21/21 [==============================] - 4s 202ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/50
21/21 [==============================] - 4s 197ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/50
21/21 [==============================] - 4s 196ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/50
21/21 [==============================] - 4s 200ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/50
21/21 [==============================] - 4s 194ms/step - loss: 0.0000e+00 - accuracy: 1.0

In [9]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Cargar el modelo
model = load_model('object_recognition_model_2.h5')

# Iniciar la cámara
cap = cv2.VideoCapture(0)

# Dimensiones de la región de interés (ROI) central
ROI_WIDTH = 400
ROI_HEIGHT = 300

# Función para preprocesar la imagen
def preprocess_image(image):
    resized_image = cv2.resize(image, (224, 224))  # Ajustar tamaño según el modelo
    preprocessed_image = resized_image / 255.0  # Normalizar
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)  # Agregar dimensión batch
    return preprocessed_image

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Obtener dimensiones del frame
    h, w, _ = frame.shape
    center_x, center_y = w // 2, h // 2  # Centro de la pantalla

    # Definir la ROI (región de interés) centrada en la imagen
    x1, y1 = center_x - ROI_WIDTH // 2, center_y - ROI_HEIGHT // 2
    x2, y2 = center_x + ROI_WIDTH // 2, center_y + ROI_HEIGHT // 2
    roi = frame[y1:y2, x1:x2]

    # Preprocesar la ROI
    preprocessed_image = preprocess_image(roi)

    # Realizar la predicción en la ROI
    prediction = model.predict(preprocessed_image)
    predicted_class = np.argmax(prediction, axis=1)

    # Convertir la imagen a escala de grises y detectar bordes
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    # Aplicar dilatación para unir contornos cercanos
    kernel = np.ones((5, 5), np.uint8)
    dilated_edges = cv2.dilate(edges, kernel, iterations=2)

    # Encontrar contornos
    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar contornos pequeños
    min_contour_area = 1000
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    # Seleccionar el contorno más cercano al centro
    if filtered_contours:
        closest_contour = min(filtered_contours, key=lambda c: cv2.pointPolygonTest(c, (ROI_WIDTH // 2, ROI_HEIGHT // 2), True))

        # Dibujar el contorno y el rectángulo delimitador
        x, y, w, h = cv2.boundingRect(closest_contour)
        cv2.rectangle(roi, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.drawContours(roi, [closest_contour], -1, (0, 0, 255), 2)

    # Mostrar la etiqueta predicha
    cv2.putText(roi, f'Predicted: {predicted_class[0]}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Mostrar la ROI procesada
    cv2.imshow('ROI', roi)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar ventanas
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 31ms/step


In [7]:
import cv2
import numpy as np
import open3d as o3d
import os

# Función para cargar imágenes desde una carpeta
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
    return images

# Función para detectar y emparejar puntos clave
def detect_and_match_keypoints(img1, img2):
    # Usar SIFT para detectar puntos clave y descriptores
    sift = cv2.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2, None)
    
    # Emparejar descriptores usando FLANN
    flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5), dict(checks=50))
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    
    # Filtrar emparejamientos usando el test de razón de Lowe
    good_matches = [m for m, n in matches if m.distance < 0.7 * n.distance]
    
    # Verificar que haya suficientes emparejamientos
    if len(good_matches) < 8:  # Se necesitan al menos 8 puntos para calcular la matriz fundamental
        return None, None
    
    # Obtener las coordenadas de los puntos clave emparejados
    points1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches])
    points2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches])
    
    return points1, points2

# Función para realizar fotogrametría y generar la nube de puntos
def photogrammetry(images):
    # Definir la matriz de calibración de la cámara (K)
    fx = fy = 1000  # Distancia focal en píxeles
    cx = images[0].shape[1] / 2  # Centro de la imagen en el eje x
    cy = images[0].shape[0] / 2  # Centro de la imagen en el eje y

    K = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0, 0, 1]
    ])
    
    # Lista para almacenar todos los puntos 3D
    all_points_3d = []
    
    # Procesar pares de imágenes consecutivas
    for i in range(len(images) - 1):
        img1 = images[i]
        img2 = images[i + 1]
        
        # Detectar y emparejar puntos clave
        points1, points2 = detect_and_match_keypoints(img1, img2)
        
        # Verificar que se detectaron suficientes puntos clave
        if points1 is None or points2 is None:
            print(f"No se detectaron suficientes puntos clave entre las imágenes {i} y {i+1}.")
            continue
        
        # Calcular la matriz fundamental y la matriz esencial
        F, mask = cv2.findFundamentalMat(points1, points2, cv2.FM_RANSAC)
        
        # Verificar que la matriz fundamental se calculó correctamente
        if F is None or F.shape != (3, 3):
            print(f"No se pudo calcular la matriz fundamental entre las imágenes {i} y {i+1}.")
            continue
        
        E = K.T @ F @ K  # K es la matriz de calibración de la cámara
        
        # Recuperar la pose relativa entre las imágenes
        _, R, t, _ = cv2.recoverPose(E, points1, points2, K)
        
        # Triangulación de puntos
        points_4d_hom = cv2.triangulatePoints(
            np.eye(3, 4),  # Matriz de proyección de la primera cámara
            np.hstack((R, t)),  # Matriz de proyección de la segunda cámara
            points1.T, points2.T
        )
        
        # Convertir puntos homogéneos a 3D
        points_3d = points_4d_hom[:3] / points_4d_hom[3]
        all_points_3d.extend(points_3d.T)
    
    # Verificar que se generaron puntos 3D
    if len(all_points_3d) == 0:
        print("No se generaron puntos 3D. Verifica las imágenes y los puntos clave.")
        return None
    
    # Crear una nube de puntos con Open3D
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(np.array(all_points_3d))
    
    return point_cloud

# Cargar imágenes desde la carpeta
folder_name = 'C://Users//jufeg//Documents//VSC_Proyecto_Grado//audifonos'
images = load_images_from_folder(folder_name)

# Realizar fotogrametría y obtener la nube de puntos
point_cloud = photogrammetry(images)

# Guardar la nube de puntos en un archivo .ply
if point_cloud is not None:
    o3d.io.write_point_cloud("output.ply", point_cloud)
    print("Nube de puntos guardada en output.ply")
    
    # Visualizar la nube de puntos
    o3d.visualization.draw_geometries([point_cloud])
else:
    print("No se pudo generar la nube de puntos.")

No se pudo calcular la matriz fundamental entre las imágenes 45 y 46.
No se pudo calcular la matriz fundamental entre las imágenes 46 y 47.
No se detectaron suficientes puntos clave entre las imágenes 56 y 57.
No se pudo calcular la matriz fundamental entre las imágenes 67 y 68.
No se pudo calcular la matriz fundamental entre las imágenes 68 y 69.
No se detectaron suficientes puntos clave entre las imágenes 78 y 79.
No se detectaron suficientes puntos clave entre las imágenes 89 y 90.
No se detectaron suficientes puntos clave entre las imágenes 111 y 112.
No se pudo calcular la matriz fundamental entre las imágenes 134 y 135.
No se pudo calcular la matriz fundamental entre las imágenes 235 y 236.
No se pudo calcular la matriz fundamental entre las imágenes 268 y 269.
No se detectaron suficientes puntos clave entre las imágenes 311 y 312.
No se detectaron suficientes puntos clave entre las imágenes 322 y 323.
No se pudo calcular la matriz fundamental entre las imágenes 323 y 324.
Nube d